# Phi-4 Mini ONNX Parallel Function Calling Tutorial

Deze notebook laat zien hoe je Phi-4 Mini kunt gebruiken met ONNX Runtime GenAI voor parallelle functie-aanroepen. Functie-aanroepen stellen het model in staat om op intelligente wijze externe tools en API's aan te roepen op basis van gebruikersverzoeken.

## Overzicht

In deze tutorial leer je:
- Phi-4 Mini instellen met ONNX Runtime GenAI
- Functieschema's definiëren voor het boeken van vluchten en hotels
- Geleide generatie gebruiken met Lark-grammatica voor gestructureerde output
- Parallelle functie-aanroepen uitvoeren voor complexe reisboekingsscenario's

## Vereisten

Voordat je deze notebook uitvoert, zorg ervoor dat je:
- Het Phi-4 Mini ONNX-model hebt gedownload
- Het pakket `onnxruntime-genai` hebt geïnstalleerd
- Basiskennis hebt van concepten rondom functie-aanroepen


## Stap 1: Vereiste bibliotheken importeren

Eerst importeren we de benodigde bibliotheken voor onze functie-aanroepimplementatie.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## Stap 2: Modelinstelling en configuratie

Nu gaan we het Phi-4 Mini ONNX-model configureren. Zorg ervoor dat je het pad vervangt door de daadwerkelijke map van je model.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## Stap 3: Configuratie van generatieparameters

Stel de generatieparameters in om het uitvoergedrag van het model te sturen. Deze instellingen zorgen voor deterministische en gerichte antwoorden bij het aanroepen van functies.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## Stap 4: Beschikbare functies definiëren

Hier definiëren we de functies die onze AI-assistent kan aanroepen. In dit voorbeeld hebben we twee reisgerelateerde functies:
1. **booking_flight_tickets**: Voor het boeken van vluchten tussen luchthavens
2. **booking_hotels**: Voor het reserveren van hotelaccommodaties

De functiedefinities volgen het schemaformaat voor functieaanroepen van OpenAI.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## Stap 5: Hulpfuncties voor grammatica-generatie

Deze hulpfuncties zetten onze functiedefinities om in Lark-grammaticaformaat, dat wordt gebruikt door ONNX Runtime GenAI voor begeleide generatie. Dit zorgt ervoor dat het model geldige functieaanroepen in het juiste JSON-formaat produceert.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## Stap 6: Test grammatica-generatie

Laten we onze functies voor grammatica-generatie testen om te zien hoe ze onze tooldefinities omzetten naar het juiste formaat.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## Stap 7: Bereid het systeemprompt en de generator voor

Nu gaan we het systeemprompt maken dat het model informeert over beschikbare tools en de generator instellen met begeleide generatieparameters.


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## Stap 8: Initialiseer Generator met Geleide Generatie

Nu gaan we de generator maken met onze geconfigureerde parameters en de Lark-grammatica toepassen voor geleide generatie.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## Stap 9: Test Parallel Functieaanroepen

Laten we nu onze setup testen met een complexe reisscenario waarin meerdere functies moeten worden aangeroepen.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### Genereer Functieaanroepen

Het model zal nu gestructureerde functieaanroepen genereren op basis van onze gebruikersaanvraag. Dankzij begeleide generatie zal de output in geldig JSON-formaat zijn dat direct kan worden uitgevoerd.

**Verwachte Output**: Het model zou functieaanroepen moeten genereren voor:
1. `booking_flight_tickets` met details van Beijing (PEK) naar Parijs (CDG)
2. `booking_hotels` met accommodatiegegevens in Parijs

Voer de onderstaande cel uit om de live generatie te zien:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## Conclusie

🎉 **Gefeliciteerd!** Je hebt met succes parallelle functie-aanroepen geïmplementeerd met Phi-4 Mini en ONNX Runtime GenAI.

### Wat Je Hebt Geleerd:

1. **Modelconfiguratie**: Hoe je Phi-4 Mini instelt met ONNX Runtime GenAI
2. **Functiedefinitie**: Hoe je functieschema's definieert voor AI-functieaanroepen
3. **Geleide Generatie**: Hoe je Lark-grammatica gebruikt voor gestructureerde outputgeneratie
4. **Parallelle Functie-aanroepen**: Hoe je complexe verzoeken afhandelt die meerdere functie-aanroepen vereisen

### Belangrijke Voordelen:

- ✅ **Gestructureerde Output**: Geleide generatie zorgt voor geldige JSON-functieaanroepen
- ✅ **Parallelle Verwerking**: Meerdere functie-aanroepen in één verzoek afhandelen
- ✅ **Hoge Prestaties**: ONNX Runtime biedt geoptimaliseerde inferentie
- ✅ **Flexibel Schema**: Functiedefinities eenvoudig toevoegen of aanpassen

### Bronnen:

- [Phi-4 Mini Documentatie](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI Documentatie](https://onnxruntime.ai/docs/genai/)
- [Best Practices voor Functie-aanroepen](https://platform.openai.com/docs/guides/function-calling)



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor kritieke informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
